In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [3]:
# Reading the data
ds=pd.read_csv("/content/spam.csv",encoding='latin')

In [4]:
# Preview data
ds.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
ds.sample(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
2685,spam,URGENT! We are trying to contact U. Todays dra...,NaN,NaN,NaN
209,ham,Both :) i shoot big loads so get ready!,NaN,NaN,NaN
630,ham,Please dont say like that. Hi hi hi,NaN,NaN,NaN
2262,ham,It should take about &lt;#&gt; min,NaN,NaN,NaN
824,ham,Have a good evening! Ttyl,NaN,NaN,NaN
5225,ham,Smile in Pleasure Smile in Pain Smile when tro...,NaN,NaN,NaN
2502,ham,Ola would get back to you maybe not today but ...,NaN,NaN,NaN
1805,ham,God bless.get good sleep my dear...i will pray!,NaN,NaN,NaN
3082,ham,K..k:)how about your training process?,NaN,NaN,NaN
4577,spam,Urgent! call 09066350750 from your landline. Y...,NaN,NaN,NaN


In [6]:
ds.shape

(5572, 5)

In [7]:
print("Rows--->",ds.shape[0])
print("Columns--->",ds.shape[1])

Rows---> 5572
Columns---> 5


In [8]:
ds.isnull()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,False,False,True,True,True
1,False,False,True,True,True
2,False,False,True,True,True
3,False,False,True,True,True
4,False,False,True,True,True
...,...,...,...,...,...
5567,False,False,True,True,True
5568,False,False,True,True,True
5569,False,False,True,True,True
5570,False,False,True,True,True


In [9]:
ds.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [10]:
ds.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [11]:
ds.isna().mean()

v1            0.000000
v2            0.000000
Unnamed: 2    0.991027
Unnamed: 3    0.997846
Unnamed: 4    0.998923
dtype: float64

In [12]:
ds.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=0,inplace=True)

In [13]:
ds

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [14]:
ds.shape

(5572, 2)

In [15]:
ds.columns=['spam/ham','SMS']

In [16]:
ds.loc[ds['spam/ham'] == 'spam', 'spam/ham',] = 0
ds.loc[ds['spam/ham'] == 'ham', 'spam/ham',] = 1

In [17]:
ds

,spam/ham,SMS
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will Ì_ b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


**Train the model with spam dataset**

In [18]:
x=ds.SMS
x

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: SMS, Length: 5572, dtype: object

In [19]:
y=ds['spam/ham']
y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: spam/ham, Length: 5572, dtype: object

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [21]:
print(x.shape)

(5572,)


In [22]:
print(x_train.shape)

(4457,)


In [23]:
print(x_test.shape)

(1115,)


In [24]:
print(y_train.shape)

(4457,)


In [25]:
print(y_test.shape)

(1115,)


In [26]:
x_train,x_test

(3890    No problem. We will be spending a lot of quali...
 5553                          Hahaha..use your brain dear
 4366    I like dis sweater fr mango but no more my siz...
 3968    Thanks for your message. I really appreciate y...
 3771                Does uncle timi help in clearing cars
                               ...                        
 3335                         Then u go back urself lor...
 1099    Been up to ne thing interesting. Did you have ...
 2514        Ok ill send you with in  &lt;DECIMAL&gt;  ok.
 3606                         I have no idea where you are
 2575                 Now thats going to ruin your thesis!
 Name: SMS, Length: 4457, dtype: object,
 5086    Omg if its not one thing its another. My cat h...
 2120    Argh my 3g is spotty, anyway the only thing I ...
 2318                            On the way to office da..
 2917                        Thanks chikku..:-) gud nyt:-*
 1352    Let Ur Heart Be Ur Compass Ur Mind Ur Map Ur S...
               

In [27]:
y_train,y_test

(3890    1
 5553    1
 4366    1
 3968    1
 3771    1
        ..
 3335    1
 1099    1
 2514    1
 3606    1
 2575    1
 Name: spam/ham, Length: 4457, dtype: object,
 5086    1
 2120    1
 2318    1
 2917    1
 1352    1
        ..
 884     1
 3821    1
 1066    1
 208     1
 1378    1
 Name: spam/ham, Length: 1115, dtype: object)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
feat_vect=TfidfVectorizer(stop_words='english',lowercase=True)
feat_vect

TfidfVectorizer(stop_words='english')

In [29]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [30]:
x_train_vec =feat_vect.fit_transform(x_train)

In [31]:
x_test_vec =feat_vect.transform(x_test)
print(x_train)

3890    No problem. We will be spending a lot of quali...
5553                          Hahaha..use your brain dear
4366    I like dis sweater fr mango but no more my siz...
3968    Thanks for your message. I really appreciate y...
3771                Does uncle timi help in clearing cars
                              ...                        
3335                         Then u go back urself lor...
1099    Been up to ne thing interesting. Did you have ...
2514        Ok ill send you with in  &lt;DECIMAL&gt;  ok.
3606                         I have no idea where you are
2575                 Now thats going to ruin your thesis!
Name: SMS, Length: 4457, dtype: object


In [32]:
x_train_vec
print("x_train--->",x_train_vec)
print("x_test---->",x_test_vec)

x_train--->   (0, 6566)	0.29071829686789585
  (0, 5260)	0.5087678982336444
  (0, 4010)	0.4050685955975014
  (0, 6073)	0.5665799184805557
  (0, 5169)	0.4141795823412651
  (1, 2116)	0.3523355719340784
  (1, 1423)	0.5940444343697558
  (1, 6859)	0.4124100686677272
  (1, 3141)	0.5940444343697558
  (2, 3565)	0.40508027557608417
  (2, 5897)	0.440107047813195
  (2, 4140)	0.440107047813195
  (2, 2845)	0.3382880530026162
  (2, 6332)	0.440107047813195
  (2, 2258)	0.3104925886287087
  (2, 3910)	0.20972094634778068
  (3, 2105)	0.17893490557083572
  (3, 7191)	0.2812743655569581
  (3, 1763)	0.2431679156261037
  (3, 6620)	0.2114722422733664
  (3, 6461)	0.2757210982911898
  (3, 7045)	0.20776132213481782
  (3, 4865)	0.26062868324883703
  (3, 2251)	0.2952674814882996
  (3, 5175)	0.3101007821209402
  :	:
  (4451, 4355)	0.5653315538937795
  (4452, 6856)	0.8442476281269602
  (4452, 4003)	0.5359533024452802
  (4453, 7234)	0.3927126429320925
  (4453, 4634)	0.3513960490954625
  (4453, 4498)	0.36145784624804855

In [33]:
model=LogisticRegression()

In [34]:
model.fit(x_train_vec,y_train)

LogisticRegression()

In [35]:
model.score(x_train_vec,y_train)

0.971729863136639

In [36]:
model.score(x_test_vec,y_test)

0.9560538116591928

In [37]:
predict_model=model.predict(x_test_vec)
predict_model

array([1, 1, 1, ..., 1, 1, 1])

In [38]:
accuracy_score(y_test,predict_model)

0.9560538116591928

In [39]:
confusion_matrix(y_test,predict_model)

array([[109,  49],
       [  0, 957]])

In [40]:
print(classification_report(y_test,predict_model))

              precision    recall  f1-score   support

           0       1.00      0.69      0.82       158
           1       0.95      1.00      0.98       957

    accuracy                           0.96      1115
   macro avg       0.98      0.84      0.90      1115
weighted avg       0.96      0.96      0.95      1115

